Import libraries

In [1]:

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
from typing import Dict, List, Optional
import ast
import itertools
import re

Import the dataset

In [2]:
df = pd.read_csv("ai_and_ml_jobs.csv")
df.head()

,Title,Company,Location,Type of Positions,Job Description,Salary,Identified_Skills
0,"Staff Applied Scientist, Marketplace",ThredUp Inc.,CA,Full-time,\nAbout thredUP thredUP is transforming resale...,190000.0,"['python', 'machine learning', 'aws', 'sql']"
1,Prompt Engineer for Generative AI (chatbot and...,Vicarious Talent Agency,WA,Full-time,"We are Vicarious, a talent agency that represe...",70000.0,"['python', 'machine learning', 'deep learning'..."
2,Data Systems Analyst/Architect,General Dynamics Information Technology,Remote,Full-time,Clearance Level None Category Data Science Loc...,104000.0,"['machine learning', 'aws', 'azure', 'nosql', ..."
3,AIOps Principle Data Scientist,CVS Health,CT,Full-time,\nBring your heart to CVS Health. Every one of...,140000.0,"['python', 'machine learning', 'aws', 'gcp']"
4,Sr. Data Scientist,Altak Group,Remote,Full-time,Job Summary:\nWe are seeking an experienced Da...,136000.0,"['python', 'machine learning', 'tableau', 'aws..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692 entries, 0 to 691
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              692 non-null    object 
 1   Company            692 non-null    object 
 2   Location           692 non-null    object 
 3   Type of Positions  692 non-null    object 
 4   Job Description    692 non-null    object 
 5   Salary             692 non-null    float64
 6   Identified_Skills  692 non-null    object 
dtypes: float64(1), object(6)
memory usage: 38.0+ KB


In [4]:
df["Identified_Skills"].value_counts(dropna=False)

Identified_Skills
['python', 'pandas', 'machine learning', 'aws', 'azure', 'docker', 'gcp', 'nosql', 'etl', 'sql', 'airflow', 'git', 'kafka', 'pyspark']    284
[]                                                                                                                                         58
['machine learning']                                                                                                                       22
['python', 'machine learning', 'sql']                                                                                                      20
['tensorflow', 'pytorch', 'python', 'machine learning', 'deep learning']                                                                   14
                                                                                                                                         ... 
['python', 'aws', 'etl', 'sql', 'git']                                                                                            

Create new column called 'tech_skills' that extracts technical skills from the column 'Identified_Skills' separated by comma


In [5]:
def parse_skills_cell(cell) -> List[str]:
    """Return a cleaned list of skills (lowercase, trimmed) from a cell value."""
    if pd.isna(cell):
        return []
    s = str(cell).strip()
    if s in ("[]", ""):
        return []
    # Try literal eval (safe for Python-list formatted strings)
    try:
        val = ast.literal_eval(s)
        if isinstance(val, (list, tuple)):
            items = [str(x).strip().lower() for x in val if x is not None and str(x).strip() != ""]
        else:
            # Not a list, fallback to comma-split
            items = [p.strip().lower() for p in s.split(",") if p.strip()]
    except Exception:
        # Fallback: strip brackets/quotes then comma-split
        s2 = s.strip().lstrip("[").rstrip("]")
        items = [p.strip().strip("'\"").lower() for p in s2.split(",") if p.strip()]
    # dedupe preserving order
    seen = set(); out=[]
    for it in items:
        if it not in seen:
            seen.add(it); out.append(it)
    return out

# apply to dataframe
df['skills_list'] = df['Identified_Skills'].apply(parse_skills_cell)

In [ ]:
# ensure an id column exists
if 'row_id' not in df.columns:
    df['row_id'] = df.index.astype(int)

# explode to long format
df_long = df.assign(skill=df['skills_list']).explode('skill')

# remove empty skills (rows originally had [])
df_long = df_long[df_long['skill'].notna() & (df_long['skill'] != "")].reset_index(drop=True)

# optional: convert to category to save memory
df_long['skill'] = df_long['skill'].astype('category')

# example counts: how many rows mention each skill
skill_counts = df_long.groupby('skill')['row_id'].nunique().sort_values(ascending=False)
print(skill_counts.head(50))

# export for Tableau
df_long['skill'].to_csv('./output/tech_skills_2.csv', index=False)


skill
python              548
machine learning    532
sql                 482
aws                 382
azure               347
gcp                 320
docker              316
etl                 313
git                 311
kafka               307
airflow             303
nosql               298
pandas              298
pyspark             289
tableau              85
deep learning        79
tensorflow           77
pytorch              74
hadoop               46
numpy                11
apache spark         10
mysql                 8
postgresql            2
spark ml              1
Name: row_id, dtype: int64


C:\Users\User\AppData\Local\Temp\ipykernel_70476\2878879640.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  skill_counts = df_long.groupby('skill')['row_id'].nunique().sort_values(ascending=False)


: 